# Projeto 3 de Ciência dos Dados

### Integrantes:


Ana Clara Carneiro

João Pedro Varella

João Guilherme Almeida

## Imports

In [ ]:
import pandas as pd
import ast
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.datasets import load_digits
digits = load_digits()

___

## Contextualização

### Pergunta:

A pergunta principal que será respondida no projeto é como diversos fatores influenciam na bilheteria de um filme. Como forma de estudo, estipularemos uma meta de bilheteria no decorrer das análises.

Sabe-se que anualmente, centenas de filmes entram em cartaz pelos diversos países do mundo. Dentre eles, a grande maioria não atinge altos valores de bilheteria. Por meio desse estudo, será possível compreender o que majoritariamente influencia se um filme será um sucesso ou se será um fracasso de bilheteria com as análises das diversas variáveis do Dataset. Essas questões serão abordadas no andamento do projeto, para que, no fim, possamos responder à pergunta inicial.

### Sobre o Dataset:

O Dataset utilizado, entitulado "The Movies Dataset", extraído do Kaggle, reúne diversas informações sobre mais de 45 mil filmes. Dentre essas, pode-se citar: atores, diretores, produtores, orçamento, bilheteria, gênero, popularidade, faixa etária, etc. 

No entanto, filtrou-se algumas variáveis julgadas como mais relevantes para o intuito do estudo. Dessa forma, certas categorias foram dispensadas para o andamento das análises.

O Dataset escolhido está disponível no link abaixo:

https://www.kaggle.com/rounakbanik/the-movies-dataset

### Método utilizado:

Para a realização do projeto utilizaremos do método de classificação do Random Forest. Afinal, o modelo se utilizará das variáveis de entrada do nosso Dataset, devolvendo, ao fim, uma variável de saída binária (no caso do nosso projeto, se o filme atinge ou não a meta estipulada). A forma pela qual esse método realiza essa análise é: Inicialmente, recebe um filme qualquer e verifica se o dado filme possui uma primeira variável de entrada (Tomemos a presença do ator Tom Hanks como exemplo) e em seguida, checa alguma outra variável de entrada (filme ser ou não de ação) e assim por diante, de forma que cada "árvore" devolverá uma variável binária para o filme atingir a meta, baseado nas respostas das variáveis de entrada. Ao fim da análise de todas as árvores, o Random Forest devolve uma variável de saída também na forma binária, se o filme conseguirá ou não bater a marca estabelecida, baseado nas respostas finais de cada árvore.

##

## Lendo arquivos

In [ ]:
df = pd.read_csv("movies_metadata.csv")
df2 = pd.read_csv("credits.csv")

## Limpeza do DataFrame "Meta Data"

### Retirada dos JSONs

No dataset utilizado, algumas variáveis estavam codificadas na forma de JSON. Para acessá-las, foi necessário extrair as informações e transformá-las em um dicionário. Nesta etapa da análise, extraímos os nomes dos atores de cada um dos filmes. Dessa forma, foi possível criar um dicionário relacionando duas variáveis, em que a chave corresponde ao nome do ator, e o valor dessa chave é o número de filmes em que cada ator atuou.

In [ ]:
from collections import defaultdict
atores = defaultdict(int)
for e in df2.index:
    data_dict = ast.literal_eval(df2["cast"][e])
    for palavras in data_dict:
        nome = palavras['name']
        
        atores[nome] += 1

### Código para limpeza de certas variáveis tais como: revenue, budget, produtora e gênero:

In [ ]:
df_0 = df[df['revenue'] > 0]
df_budget = df_0[df_0['budget'] != '0']
df_clean = df_budget[df_budget['production_companies'] != '[]']
df_super_clean = df_clean[df_clean['genres'] != '[]']

In [ ]:
df_mega_clean = df_super_clean[["id","genres","budget","revenue","production_companies"]]
df_mega_clean['budget'] = df_mega_clean['budget'].astype(float)
df_mega_clean.head()

---

## Análise Exploratória

Primeiramente foram analisadas as variáveis mais simplórias do DataFrame. Dessa forma, foram plotados histogramas que relacionam a frequência absoluta das variáveis Budget e Revenue.

### Histograma da variável budget

In [ ]:
df_mega_clean.budget.plot.hist(density = False)
plt.title('Budget')
plt.show()

##### Pode-se observar pelo histograma acima que dentre os filmes disponibilizados no Dataset, a esmagadora maioria possui um baixo orçamento. Este fato explicita que os grandes filmes orçamentários representam somente uma pequena fatia de todos os filmes produzidos

----

### Histograma da variável revenue

In [ ]:
df_mega_clean.revenue.plot.hist(density = False)
plt.title('Revenue')
plt.show()

##### Neste segundo histograma, que relaciona a bilheteria dos filmes, nota-se que, novamente, a esmagadora minoria atingiu altos valores nesse quesito. Isto demonstra que dentre todos os filmes anualmente chegam aos cinemas, poucos deles atingem um grande sucesso

----

### Gráfico de dispersão das variáveis budget e revenue

##### Neste gráfico de dispersão, foram relacionadas as duas variáveis acima, de modo que fosse possível constatar a relação entre filmes com alto orçamento e filmes com alta bilheteria.

In [ ]:
df_mega_clean.plot.scatter(x="budget", y = "revenue")
plt.title('Budget x Revenue')
plt.show()

In [ ]:
generos = []
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    for palavras in data_dict:
        generos.append(palavras['name'])

df_generos = pd.DataFrame(generos)
df_generos
geneross = df_generos[0].value_counts()
geneross
geneross.plot(kind='bar')

In [ ]:
revenues_por_categoria = {}
categorias = [
    'Drama', 'Comedy', 'Thriller',
    'Action','Romance','Adventure','Crime','Science Fiction',
    'Horror','Family','Fantasy','Mystery','Animation','History',
    "War",'Music','Western','Documentary','Foreign','TV Movie'
]

for c in categorias:
    revenues_por_categoria[c] = []

for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    for palavras in data_dict:
        cat = palavras["name"]
        #print(cat)
        if cat in revenues_por_categoria:
            revenues_por_categoria[cat].append(df_mega_clean['revenue'][e])
            
for k in revenues_por_categoria:
    revenues_por_categoria[k] = (np.array(revenues_por_categoria[k]))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.boxplot(revenues_por_categoria.values(), showfliers=False)
ax.set_xticklabels(revenues_por_categoria.keys(), rotation='vertical');
#plt.yscale('log')
plt.show()

In [ ]:
produtoras = []
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["production_companies"][e])
   # print(data_dict)
    for palavras in data_dict:
        produtoras.append(palavras['name'])

df_produtoras = pd.DataFrame(produtoras)
df_produtoras
produtorass = df_produtoras[0].value_counts().head(30)
produtorass
produtorass.plot(kind='bar')

In [ ]:
revenues_por_produtora = {}
produtoras = [
    'Warner Bros.','Universal Pictures','Paramount Pictures','Twentieth Century Fox Film Corporation','Columbia Pictures',
    'New Line Cinema','Metro-Goldwyn-Mayer (MGM)','Touchstone Pictures','Walt Disney Pictures','Columbia Pictures Corporation',
    'Relativity Media','United Artists','Miramax Films','TriStar Pictures','Canal+','Village Roadshow Pictures','DreamWorks SKG',
    'Regency Enterprises','Lionsgate','Amblin Entertainment','Summit Entertainment','Dune Entertainment','Fox Searchlight Pictures',
    'Dimension Films','Working Title Films','Fox 2000 Pictures','StudioCanal','Silver Pictures','Hollywood Pictures',
    'The Weinstein Company'
]

for p in produtoras:
    revenues_por_produtora[p] = []

for f in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["production_companies"][f])
    #print(data_dict)
    for palavras in data_dict:
        #print(palavras)
        prod = palavras["name"]
        if prod in revenues_por_produtora:
            revenues_por_produtora[prod].append(df_mega_clean['revenue'][f])
        
            
#revenues_por_produtora
            
for k in revenues_por_produtora:
    revenues_por_produtora[k] = (np.array(revenues_por_produtora[k]))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.boxplot(revenues_por_produtora.values(), showfliers=False)
ax.set_xticklabels(revenues_por_produtora.keys(), rotation='vertical');
#plt.yscale('log')
plt.show()

## Limpeza do DataFrame "Credits"

In [ ]:
ser_atores = pd.Series(atores)
ser_atores.head()

Para visualizar de maneira mais ordenada e clara os dados do DataFrame, selecionamos a parcela de 0.1% de atores com mais produções no cinema. Dessa forma, atingimos todos com mais de 28 aparições:

In [ ]:
ser_atores = ser_atores[ser_atores > 28].sort_values(ascending=False)

Para utilizar os dados do DataFrame acima no método do Random Forrest, foi necessário transformar as informações nele em um outro DataFrame com solução binária.

Nesta nova tabela, cada linha representa cada um dos filmes contidos no database, e cada coluna representa os atores selecionados previamente:

In [ ]:
df_filmes_atores = pd.DataFrame(index=df2.index, columns=ser_atores.index, dtype=np.uint8)
for e in df2.index:
    data_dict = ast.literal_eval(df2["cast"][e])
    for palavras in data_dict:
        nome = palavras['name']
        if nome in df_filmes_atores.columns:
            df_filmes_atores[nome][e] = 1

In [ ]:
df_filmes_atores[df_filmes_atores != 1] = 0
df_filmes_atores.head()

Nota-se que neste DataFrame, a grande maioria dos dígitos é 0, o que representa o óbvio, os atores apareceram em poucos filmes em relação ao total estudado

In [ ]:
df.drop_duplicates(subset ="original_title", 
                     keep = 'first', inplace = True) 
df["original_title"].value_counts().value_counts()

In [ ]:
df['bilhao'] = df.revenue > 1e9

## Limpeza das variáveis de entrada tais como: revenue, budget, produtora e gênero:

In [ ]:
df_0 = df[df['revenue'] > 0]
df_budget = df_0[df_0['budget'] != '0']
df_clean = df_budget[df_budget['production_companies'] != '[]']
df_super_clean = df_clean[df_clean['genres'] != '[]']

In [ ]:
df_mega_clean = df_super_clean[["id","genres","budget","revenue","production_companies"]]
df_mega_clean['budget'] = df_mega_clean['budget'].astype(float)

In [ ]:
df_mega_clean.revenue.quantile(0.8)

Na célula acima, verificou-se que o valor que corta os 20% das maiores bilheterias é 129 milhões de dólares, dessa forma, definimos que o valor que seria usado no modelo como meta de bilheteria para os filmes seria 150 milhões de dólares

In [ ]:
milhao = []
for dinheiro in df_mega_clean.revenue:
    if dinheiro > 1.5e8:
        milhao.append(1)
    else:
        milhao.append(0)

Após definir a meta do modelo, percorremos todos os filmes e adicionamos uma nova coluna ao nosso dataframe original que indica se cada filme atingiu ou não atingiu os 150 milhões de dólares

In [ ]:
df_mega_clean['Fez ou não 150 mi'] = milhao
df_mega_clean.head()

### Filtrando os atores por meio da limpeza estabelecida no DataFrame "MetaData"

In [ ]:
lista_teste = []
for numero in df_filmes_atores.index:
    if numero not in df_mega_clean.id:
        lista_teste.append(numero)
        
atores_certos = df_filmes_atores.drop(lista_teste, axis=0)

No dataframe acima, foi selecionado os atores que participam dos filmes filtrados no Dataframe "mega_clean" fazendo com que agora podemos observar quais atores participaram dos filmes relevantes 

In [ ]:
atores_certos['Fez ou não 150 mi'] = milhao
atores_certos.head()

In [ ]:
sim_ou_nao = atores_certos['Fez ou não 150 mi']
y = df_mega_clean['Fez ou não 150 mi']

---

# Randon Forest 

O próximo passo será nós rodarmos o Randon Forest com o nossos Datframes "binários" para que agora possamos verificar nosso objetivo que é verificar se de fato as feature que nós escolhemos impactam na bilheteria

## Influência dos atores na bilheteria

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(atores_certos.drop(['Fez ou não 150 mi'],axis='columns'),sim_ou_nao,test_size = 0.3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 40)
model.fit(X_train,Y_train)

In [ ]:
model.score(X_test,Y_test)

Dado que nossa acurácia é aproximadamente 80%, é possível afirmar que nosso classificador não é competente de classificar se um filme passará ou não da marca de 150 milhões de doláres se embasando apenas nos atores. Isso se dá pelo fato de apenas 20% dos filmes serem capazes de passar da nossa marca, portanto o nosso classificador não é capaz de ser melhor de um outro que julga como todos os filmes jamais passarão de 150 milhões.

### Relevância das Categorias 

In [ ]:
categorias2 = defaultdict(int)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    for palavras in data_dict:
        tipo = palavras['name']
        categorias2[tipo] += 1

In [ ]:
ser_categorias = pd.Series(categorias2)

In [ ]:
df_filmes_categorias = pd.DataFrame(index=df_mega_clean.index, columns=ser_categorias.index, dtype=np.uint8)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    #print(data_dict)
    for palavras in data_dict:
        tipo = palavras['name']
        #print(tipo)
        if tipo in df_filmes_categorias.columns:
             df_filmes_categorias[tipo][e] = 1


In [ ]:
df_filmes_categorias[df_filmes_categorias != 1] = 0
df_filmes_categorias.head()

In [ ]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(df_filmes_categorias,sim_ou_nao,test_size = 0.3)

model = RandomForestClassifier(n_estimators = 40)
model.fit(X2_train,Y2_train)

In [ ]:
model.score(X2_test,Y2_test)

Pela mesma justificativa da relevância dos atores, podemos concluir que nosso classificador usando categorias de filme não não é capaz de dizer se um filme fará ou não 150 milhões de dólares na bilheteria. Isso se da pelo fato de a acurácia do classificador com "categorias" como variável de entrada ter dado, também, aproximadamente 80%. A diferença dos atores para as categorias de filme é apenas que a acurácia do segundo ter dado um pouco mais, ou seja, o segundo é um classificador mais preciso, o que indica que "categorias de filmes" é mais relevante pare dizer se um filme passará ou não da nossa marca

## Influência das produtoras na bilheteria

In [ ]:
prod2 = defaultdict(int)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean['production_companies'][e])
    for palavras in data_dict:
        produ = palavras['name']
        prod2[produ] += 1

In [ ]:
ser_produtora = pd.Series(prod2)

In [ ]:
df_filmes_produtoras = pd.DataFrame(index=df_mega_clean.index, columns=ser_produtora.index, dtype=np.uint8)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["production_companies"][e])
    #print(data_dict)
    for palavras in data_dict:
        produ = palavras['name']
        #print(tipo)
        if produ in df_filmes_produtoras.columns:
             df_filmes_produtoras[produ][e] = 1


In [ ]:
df_filmes_produtoras[df_filmes_produtoras != 1] = 0
df_filmes_produtoras.head()

In [ ]:
X3_train, X3_test, Y3_train, Y3_test = train_test_split(df_filmes_produtoras,sim_ou_nao,test_size = 0.3)

model = RandomForestClassifier(n_estimators = 40)
model.fit(X3_train,Y3_train)

In [ ]:
model.score(X3_test,Y3_test)

Pode-se afirmar que nosso classificador que recebe como variável de entrada "produtoras de filmes" não é muito bem sucedido. Isso se dá pelas mesmas justificativas expressas anteriormente, ou seja, como a acurácia deu aproxidamente 85%(próximo de 80), esse classificador não pode ser avaliado como um melhor que um classificador que julga que nenhum filme passará da marca de 150 milhões de dólares, já que a acurácia do segundo deveria ser 80, estando muito próximo da acurácia do primeiro. Dessa forma, um classificador ruim (que julga que nenhum filme passará de nossa marca) tem uma acurácia próxima à do nosso classificador, fazendo o segundo um classificador ruim também. Mesmo assim, até agora, a feature que possui maior relevância para nosso classificador, pode-se assumir que é "produtoras de filmes" já que essa deu a maior acurácia

## Influência do orçamento na bilheteria

In [ ]:
df_Bud = pd.DataFrame(df_mega_clean.budget)
df_Bud.head()

In [ ]:
X5_train, X5_test, Y5_train, Y5_test = train_test_split(df_Bud,sim_ou_nao,test_size = 0.3)

model = RandomForestClassifier(n_estimators = 60)
model.fit(X5_train,Y5_train)

In [ ]:
model.score(X5_test,Y5_test)

In [ ]:
sorted(list(zip(model.feature_importances_, X5_test.columns)), reverse=True)

Por fim, nossa classificador que utiliza como variável de entrada "orçamento" também não pode ser expresso como classificador bom. Isso pois, este apresenta uma acurácia de aproximadamente 87%, de novo, próximo à 80%, fazendo com que este tenha uma acurácia tão boa quanto um classificador ruim que julga que nenhum filme passa da marca de 150 milhões de dólares. Porém, esse classificador indica que a feature "orçamento" é a mais relevante dentre todas nossas features, já que este apresenta a maior acurácia.

## Random Forest com todas as features

In [ ]:
df_forest = pd.concat([
    df_filmes_categorias,
    atores_certos.drop(['Fez ou não 150 mi'],axis='columns'),
    df_filmes_produtoras,
    df_mega_clean.budget,

], axis=1)

In [ ]:
X4_train, X4_test, Y4_train, Y4_test = train_test_split(df_forest,sim_ou_nao,test_size = 0.3)

model = RandomForestClassifier(n_estimators = 60)
model.fit(X4_train,Y4_train)

In [ ]:
model.score(X4_test,Y4_test)

In [ ]:
sorted(list(zip(model.feature_importances_, X4_test.columns)), reverse=True)

In [ ]:
model.feature_importances_

# Modelos utilizados para comparação

## Regressão Logística com todas as features

In [ ]:
model = LogisticRegression()
model.fit(X4_train,Y4_train)
Ypred = model.predict(X4_test)
print(accuracy_score(Y4_test,Ypred))

## Máquina de vetores de suporte com todas as features

In [ ]:
model = SVC()
model.fit(X4_train,Y4_train)
Ypred = model.predict(X4_test)
print(accuracy_score(Y4_test,Ypred))